## 1. Setup the game

In [ ]:
!pip install gym_super_mario_bros==7.3.0 nes_py

In [ ]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import gym

In [ ]:
from nes_py import NESEnv
_reset = NESEnv.reset

def reset(*args, **kwargs):
    obs_info = _reset(*args, **kwargs)
    obs, info = obs_info if type(obs_info) == tuple else (obs_info, {})
    return obs, info

NESEnv.reset = reset

In [ ]:
SIMPLE_MOVEMENT

In [ ]:
# Setup game
env = gym.make('SuperMarioBros2-v0', apply_api_compatibility=True, render_mode='human')
# env.metadata['render.modes'] = ['human', 'rgb_array'] 
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [ ]:
SIMPLE_MOVEMENT[env.action_space.sample()]

In [ ]:
# Create a flag - restart or not
done = True
for step in range(10000):
    if done:
        env.reset()
    action = env.action_space.sample()
    state, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    
    env.render()
env.close()

## 2. Pre-processing


In [ ]:
import numpy as np

In [ ]:
!pip3 install torch torchvision torchaudio

In [ ]:
!pip install 'stable-baselines3[extra]' #to use RL Algorithms

In [ ]:
#Import Frame Stacker Wrapper(capture couple of frames) and Gray-scaling wrapper(coloured game to grayscale version, to cut down processing time)
from gym.wrappers import FrameStack, GrayScaleObservation

#Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

#Import matplotlib
from matplotlib import pyplot as plt


In [ ]:
# 1. Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# 2. Simplify the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3. Grayscale
env = GrayScaleObservation(env, keep_dim=True)
# 4. Wrap inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# 5. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
state=env.reset()

In [ ]:
type(state)

In [ ]:
state.shape

In [ ]:
state, reward, terminated, truncated, info= env.step([env.action_space.sample()])

In [ ]:
plt.imshow(state)

## 3. Model Training

In [ ]:
# Import os for file path management
import os 
# Import PPO for algos
from stable_baselines3 import PPO
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [ ]:
# This is the AI model started
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, 
            n_steps=512) 

In [ ]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=1000000, callback=callback)

In [ ]:
model.save('thisisatestmodel')

## 4. Testing

In [ ]:
# Load model
model = PPO.load('./train/best_model_1000000')

In [ ]:
state = env.reset()

In [ ]:
# Start the game 
state = env.reset()
# Loop through the game
while True: 
    
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()